In [2]:
import glob
import pickle
import numpy as np
from music21 import converter, instrument, note, chord
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

In [3]:
def create_model(input, n_vocab):
    model = Sequential()

    model.add(LSTM(
        units=512,
        input_shape = (input.shape[1], input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))

    model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))

    model.add(LSTM(256))
    model.add(Dense(256))
    model.add(Dropout(0.3))
    
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model